# huggingface 处理大文件

1.1 本地文件处理为dataset

1.2 分块加载大文件

1.3 AI模型如何读取大文件

In [1]:
from datasets import load_dataset

/home/ec2-user/software/miniconda3/envs/week2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_files = {"train": "/home/ec2-user/project/AI/Study/Week1/data/train.csv", "validation": "/home/ec2-user/project/AI/Study/Week1/data/val.csv"}
down_dataset = load_dataset("csv", data_files=data_files)

In [13]:
down_dataset # train表 的 第一行

DatasetDict({
    train: Dataset({
        features: ['prot_id', 'seq', 'seq_len', 'pdb_filename', 'ptm', 'mean_plddt', 'emb_filename', 'label', 'source'],
        num_rows: 963
    })
    validation: Dataset({
        features: ['prot_id', 'seq', 'seq_len', 'pdb_filename', 'ptm', 'mean_plddt', 'emb_filename', 'label', 'source'],
        num_rows: 275
    })
})

1.2 分块加载大文件

In [24]:
#!pip install zstandard

In [3]:
from datasets import load_dataset

# 这需要几分钟才能运行,所以在你等待的时候去喝杯茶或咖啡 :)
data_files = "https://huggingface.co/datasets/casinca/PUBMED_title_abstracts_2019_baseline/resolve/main/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
pubmed_dataset_streamed = load_dataset(
    "json", 
    data_files=data_files, 
    split="train",
    streaming=True, # 使用streaming模式来处理大型数据集
    cache_dir="../test/dataset"
    )
#pubmed_dataset
#next(iter(pubmed_dataset_streamed))


1.3 AI模型如何读取大文件

首先要把数据集转为embedding，然后导入到模型中去训练？

In [ ]:
# 导入必备库
from datasets import load_dataset
import torch
import numpy as np
import os
import json
from transformers import AutoModel, AutoTokenizer

# ===================== 1. 基础配置（改这里的路径即可） =====================
DATA_FILE = "/home/ec2-user/project/AI/Study/Week1/data/train.csv"  # 你的CSV文件
SAVE_DIR = "/home/ec2-user/project/AI/Study/Week1/test/dataset/embedding"  # 保存embedding的文件夹
CHUNK_SIZE = 64  # 每次处理64条数据（按需改）
MODEL_NAME = "facebook/esm2_t6_8M_UR50D"  # ESM-2小模型
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # 自动选GPU/CPU

# ===================== 2. 加载模型和分词器（一次性加载） =====================
print("加载ESM-2模型...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).eval().to(DEVICE)  # 推理模式

# 创建保存文件夹
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== 3. 流式读取数据+计算embedding =====================
# 加载CSV为流式数据集（不占内存）
dataset = load_dataset("csv", data_files=DATA_FILE, split="train", streaming=True)
dataset_iter = iter(dataset)  # 转为迭代器
chunk_idx = 0  # 块编号

print("开始计算embedding...")
while True:
    # 1. 读取当前块的64条数据，把它保存到列表中
    batch_data = []
    for _ in range(CHUNK_SIZE):
        try:
            batch_data.append(next(dataset_iter))  # 读一条数据
        except StopIteration:
            break  # 数据读完，退出
    
    if not batch_data:
        break  # 没有数据了，结束循环
    
    # 2. 提取序列和ID，过滤空值
    seqs = [item["seq"] for item in batch_data if item["seq"]]
    prot_ids = [item["prot_id"] for item in batch_data if item["seq"]]
    if not seqs:
        chunk_idx += 1
        continue
    
    # 3. 分词+计算embedding
    inputs = tokenizer(seqs, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(DEVICE)
    with torch.no_grad():  # 关闭梯度，省显存
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # 取<cls>标记的embedding形状：[]批次有 64 条序列，每条序列分词后长度是 100，模型维度 320； [:, 0, :]切片：中间层不要保留64 320 【Numpy 数组是 Python 通用格式，易保存、易处理（比如转列表写入 JSON），而 PyTorch 张量无法直接序列化到文件。
    
    # 4. 保存结果（JSON格式，ID和embedding一一对应）
    emb_dict = {pid: emb.tolist() for pid, emb in zip(prot_ids, embeddings)}
    json_path = os.path.join(SAVE_DIR, f"emb_chunk_{chunk_idx}.json")
    with open(json_path, "w") as f:
        json.dump(emb_dict, f)
    
    #print(f"第{chunk_idx}块：保存{len(emb_dict)}条embedding")
    chunk_idx += 1

# ===================== 4. 加载embedding（按需使用） =====================
def load_all_embeddings(save_dir):
    """加载所有保存的embedding，返回{prot_id: embedding}的字典"""
    all_emb = {}
    for file in os.listdir(save_dir):
        if file.startswith("emb_chunk_") and file.endswith(".json"):
            with open(os.path.join(save_dir, file), "r") as f:
                all_emb.update(json.load(f))
    return all_emb

# 加载所有embedding并查看结果
all_emb = load_all_embeddings(SAVE_DIR)
print(f"\n全部加载完成！共{len(all_emb)}条embedding")
print(all_emb["REP|ywg_BAD18935.1"]) #一维矩阵，长320

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


加载ESM-2模型...


开始计算embedding...
第0块：保存64条embedding
第1块：保存64条embedding
第2块：保存64条embedding
第3块：保存64条embedding
第4块：保存64条embedding
第5块：保存64条embedding
第6块：保存64条embedding
第7块：保存64条embedding
第8块：保存64条embedding
第9块：保存64条embedding
第10块：保存64条embedding
第11块：保存64条embedding
第12块：保存64条embedding
第13块：保存64条embedding
第14块：保存64条embedding
第15块：保存3条embedding

全部加载完成！共963条embedding
目标ID REP|ywg_BAD18935.1 的embedding长度：320


In [ ]:
# 保存到本地的代码
#存为 Parquet 文件（适合后续模型训练）
import os
import pandas as pd

# 创建存放目录
save_dir = "./pubmed_embeddings"
os.makedirs(save_dir, exist_ok=True)

# 假设我们每 1000 条保存一个文件，防止内存溢出
batch_size = 1000
current_batch = []
file_count = 0

print("开始处理并保存...")

# 这里的 pubmed_with_embeddings 是我们上一段代码中 map 后的流式对象
for i, record in enumerate(pubmed_with_embeddings):
    current_batch.append(record)
    
    # 当累计到指定条数时，保存到本地
    if (i + 1) % batch_size == 0:
        df = pd.DataFrame(current_batch)
        file_path = os.path.join(save_dir, f"part_{file_count}.parquet")
        df.to_parquet(file_path)
        
        print(f"已保存 {i+1} 条数据到 {file_path}")
        current_batch = []
        file_count += 1
        
        # 仅作为演示：如果你只想测试前 5000 条，可以 break
        if file_count >= 5: 
            break

# 处理剩余的数据
if current_batch:
    df = pd.DataFrame(current_batch)
    df.to_parquet(os.path.join(save_dir, f"part_{file_count}.parquet"))

开始处理并保存...


NameError: name 'pubmed_with_embeddings' is not defined

HDF5：存储纯数字非常快，但如果你想同时存“论文标题（字符串）”、“作者（列表）”和“向量（数组）”，HDF5 会变得非常麻烦，且效率降低。